# MFCCS
Obtener los mfccs para 5 clases c/u de 500 audios, para un total de 15.000 audios

In [1]:
import librosa
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import sklearn as sk
import pandas as pd

In [2]:
PATHSAMPLES = '../data/training_list.txt'
NSAMPLES = 10
SOUNDS =np.array(['yes','no','right','five','nine'])

In [3]:
def get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS):
    ROOTPATH= 'data/'
    data = pd.read_csv(PATHSAMPLES, sep=" ", header=None)
    data = data.values #Convertir en numpy array
    labels = []
    speakers  = []
    paths = []
    for path in data:
        pathSplit =path[0].split('/')
        speakerName = pathSplit[1].split('_')[0]
        word = pathSplit[0]
        isWord = np.where(SOUNDS == word)
        if len(isWord[0]) != 0:
            label = SOUNDS[isWord[0][0]]
            labels.append(label)
            speakers.append(speakerName)
            paths.append(ROOTPATH+path[0])
    NSAMPLES = NSAMPLES
    first = labels.index(SOUNDS[0])
    last = first +NSAMPLES
    matrixMajor = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
    for i in range(1,np.size(SOUNDS,0)):
        first = labels.index(SOUNDS[i])
        last = first +NSAMPLES
        clasei = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
        matrixMajor = np.append(matrixMajor,clasei,axis=0)
    return matrixMajor

In [4]:
path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)

In [5]:
path_word_speakerId.shape

(50, 3)

In [6]:
path_word_speakerId

array([['data/yes/004ae714_nohash_0.wav', 'yes', '004ae714'],
       ['data/yes/004ae714_nohash_1.wav', 'yes', '004ae714'],
       ['data/yes/00970ce1_nohash_0.wav', 'yes', '00970ce1'],
       ['data/yes/00f0204f_nohash_0.wav', 'yes', '00f0204f'],
       ['data/yes/00f0204f_nohash_1.wav', 'yes', '00f0204f'],
       ['data/yes/00f0204f_nohash_2.wav', 'yes', '00f0204f'],
       ['data/yes/012c8314_nohash_0.wav', 'yes', '012c8314'],
       ['data/yes/0132a06d_nohash_0.wav', 'yes', '0132a06d'],
       ['data/yes/0132a06d_nohash_1.wav', 'yes', '0132a06d'],
       ['data/yes/0132a06d_nohash_2.wav', 'yes', '0132a06d'],
       ['data/no/012c8314_nohash_0.wav', 'no', '012c8314'],
       ['data/no/0132a06d_nohash_0.wav', 'no', '0132a06d'],
       ['data/no/0132a06d_nohash_1.wav', 'no', '0132a06d'],
       ['data/no/0132a06d_nohash_2.wav', 'no', '0132a06d'],
       ['data/no/0132a06d_nohash_3.wav', 'no', '0132a06d'],
       ['data/no/0132a06d_nohash_4.wav', 'no', '0132a06d'],
       ['data/no/013

In [29]:
def build_features(path_word_speakerId, n_mfcc,NSAMPLES, PATHSAMPLES, SOUNDS):
    N_rows = NSAMPLES*len(SOUNDS)*n_mfcc
    N_colums = n_mfcc +3# número de columnas del mfcc + 3 columnas (la palabra y el speaker id, path al que pertenece)
    # el path es necesario ya que estamos teniendo un enfoque de multiples instancias.
    path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)
    matrix_features = np.ones((1,20)) # inicialmente

    for sound_info in path_word_speakerId:
        audio_path = '../'+sound_info[0]
        x , sr = librosa.load(audio_path)
        x_normalize=sk.preprocessing.minmax_scale(x, axis=0)       
        mfccs = librosa.feature.mfcc(x_normalize, sr=sr,n_mfcc=n_mfcc,hop_length=int(0.010*sr), n_fft=int(0.025*sr))
        matrix_features = np.concatenate((matrix_features,mfccs.T), axis=0)
        
    #matrixFinal = np.array(matrix_features)
    return matrix_features[1:,:]

In [30]:
matrixFinal = build_features(path_word_speakerId, 20,NSAMPLES, PATHSAMPLES, SOUNDS)


In [31]:
matrixFinal.shape

(4979, 20)

In [32]:
matrixFinal

array([[-590.4909668 ,   26.30455017,   -3.91339087, ...,   18.9973259 ,
          15.0318222 ,   13.07897949],
       [-504.52578735,   35.24043274,  -39.94683075, ...,   22.04785919,
          13.15106201,    6.71503878],
       [-553.69812012,   39.91327286,  -17.93079758, ...,   23.83279037,
          14.30254555,    5.13244534],
       ...,
       [-683.78094482,   19.85124207,   19.82871628, ...,   17.75148392,
          17.50481415,   17.24602699],
       [-683.44195557,   20.32917786,   20.30220795, ...,   17.88118744,
          17.60183144,   17.31045723],
       [-683.70672607,   19.89104843,   19.76181221, ...,   17.68603516,
          17.49160004,   17.28886795]])

In [8]:
matrix_features = np.ones((10,12))
matrix_features[0:3]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])